# **Data Visualisation**

## Objectives

1. Find the average size of images and generate a file containing the new image size data.
2. Calculate and visually represent the average variability of the dataset for each label.
3. Evaluate the distinction in average images between the "eyes open" and "eyes closed" labels.
4. Create image montages for both "eyes open" and "eyes closed" labels.

## Inputs

* Use the awareness dataset, which comprises of .jpg files organized into three folders.
  * File paths for the folders are:
    - inputs/awareness/test
    - inputs/awareness/train
    - inputs/awareness/validation

## Outputs

* Determine the computed average image size for the train set.
* Generate a .pkl file containing the preferred image sizes used for model fitting.
* Plot the mean and variability of images per label.
* Visualize the difference between "eyes open" and "eyes closed" images.
* Produce an image montage of .png files suitable for integration into the dashboard.


---

## Import Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import tensorflow as tf
sns.set_style("white")
from matplotlib.image import imread

2023-12-29 17:11:38.021383: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [2]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/PP5-Driver-Awareness-Detector/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [3]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [4]:
current_dir = os.getcwd()
current_dir

'/workspace/PP5-Driver-Awareness-Detector'

---

##  Set the Input Directories

Assign the input directories to variables to be used

In [6]:
my_data_dir = 'inputs/awareness'

# add extension for each folder
train_data = my_data_dir + '/train' 
validation_data = my_data_dir + '/validation'
test_data = my_data_dir + '/test'

---

##  Set the Output Directories

Assign the path for the output

In [11]:
version = 'v1'
file_path = f'outputs/{version}'

if 'outputs' in os.listdir(current_dir) and version in os.listdir(current_dir + '/outputs'):
    print(f'Version already available create a new version.')
    pass
else:
    os.makedirs(name=file_path)

Version already available create a new version.
